In [2]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [10]:
# change the path to your train pictures file
train_path = './Train/';

X = []
for filename in os.listdir(train_path):
    X.append(img_to_array(load_img(train_path + filename)))
X = np.array(X, dtype=float)

# Set up train and validation data
split = int(0.90*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

Xval = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xval = Xval.reshape(Xval.shape+(1,))
Yval = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Yval = Yval / 128

In [11]:
model = Sequential()
model.add(InputLayer(input_shape=(96, 96, 1)))
model.add(Conv2D(24, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(24, (3, 3), activation='sigmoid', padding='same', strides=2))   
model.add(Conv2D(48, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(48, (3, 3), activation='sigmoid', padding='same', strides=2))   
model.add(Conv2D(96, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(96, (3, 3), activation='sigmoid', padding='same', strides=2))   
model.add(Conv2D(192, (3, 3), activation='sigmoid', padding='same'))             
model.add(Conv2D(96, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(48, (3, 3), activation='sigmoid', padding='same'))              
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(24, (3, 3), activation='sigmoid', padding='same'))              
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(12, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))               
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse')

In [12]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 5
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

print(batch_size * split)

180


In [13]:
# steps per epoc - whole dataset / batch_size
model.fit_generator(image_a_b_gen(batch_size), epochs=15, steps_per_epoch= split, validation_data=(Xtest, Ytest))
model.save('colorizing')

Epoch 1/15
36/36 [==============================] - 10s 284ms/step - loss: 0.0933 - val_loss: 0.0239
Epoch 2/15
36/36 [==============================] - 9s 240ms/step - loss: 0.0283 - val_loss: 0.0191
Epoch 3/15
36/36 [==============================] - 9s 247ms/step - loss: 0.0245 - val_loss: 0.0186
Epoch 4/15
36/36 [==============================] - 9s 242ms/step - loss: 0.0256 - val_loss: 0.0183
Epoch 5/15
36/36 [==============================] - 8s 232ms/step - loss: 0.0248 - val_loss: 0.0179
Epoch 6/15
36/36 [==============================] - 8s 228ms/step - loss: 0.0243 - val_loss: 0.0180
Epoch 7/15
36/36 [==============================] - 8s 231ms/step - loss: 0.0243 - val_loss: 0.0181
Epoch 8/15
36/36 [==============================] - 8s 228ms/step - loss: 0.0239 - val_loss: 0.0173
Epoch 9/15
36/36 [==============================] - 9s 240ms/step - loss: 0.0238 - val_loss: 0.0172
Epoch 10/15
36/36 [==============================] - 9s 241ms/step - loss: 0.0237 - val_loss: 0.017

In [14]:
color_me = []
test_path = './Test/'
for filename in os.listdir(test_path):
    color_me.append(img_to_array(load_img(test_path+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((96, 96, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

c:\users\tsofan\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\skimage\color\colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
c:\users\tsofan\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
c:\users\tsofan\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\skimage\color\colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 4 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
c:\users\tsofan\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\skimage\color\colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 2 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
c:\users\tsofan\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\